<a href="https://colab.research.google.com/github/MO230101/test/blob/main/%E3%82%B7%E3%83%B3%E3%83%9C%E3%83%AA%E3%83%83%E3%82%AF%E5%9B%9E%E5%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('Swelling_RDKit_DSC_MSEGPMGnodry_Top10_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# 特徴量のスケーリング
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# カスタム関数定義 (例: 二乗関数と立方根関数)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square,
                                     arity=1,
                                     name='square')
cbrt_function = make_function(function=protected_cbrt,
                                   arity=1,
                                   name='cbrt')

# 精度評価関数（分類）
def binary_accuracy_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    y_pred_binary = (y_pred > 0.5).astype(int)
    return accuracy_score(y_true, y_pred_binary)

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', square_function]
param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [3000, 5000],
    'generations': [300, 500],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.0, 1e-5], # 0 を追加
}

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.0, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb12e810410>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
#20250502 for swelling　スケーリング削除
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('Swelling_RDKit_DSC_MSEGPMGnodry_Top10_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function] # 関数オブジェクトを直接指定

param_grid_improved = {
    'n_features_to_select': [3, 5, 10],
    'function_set': [function_set],
    'population_size': [1000, 3000],
    'generations': [500, 1000],
    'tournament_size': [10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.01],
}

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

<ipython-input-13-652c37e4f48e>:49: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 1/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19148dd10>, <gplearn.functions._Function object at 0x7fb1368798d0>, <gplearn.functions._Function object at 0x7fb135e9d6d0>, <gplearn.functions._Function object at 0x7fb135e9e310>]}

Trial 2/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 1000, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19148dd10>, <gplearn.functions._Function object at 0x7fb1368798d0>, <gplearn.functions._Function object at 0x7fb135e9d6d0>, <gplearn.functions._Function object at 0x7fb135e9e310>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 4/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 1000, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19148dd10>, <gplearn.functions._Function object at 0x7fb1368798d0>, <gplearn.functions._Function object at 0x7fb135e9d6d0>, <gplearn.functions._Function object at 0x7fb135e9e310>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 5/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19148dd10>, <gplearn.functions._Function object at 0x7fb1368798d0>, <gplearn.functions._Function object at 0x7fb135e9d6d0>, <gplearn.functions._Function object at 0x7fb135e9e310>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.625
  Precision: 0.625
  Recall: 1.0
  F1-score: 0.7692307692307693
  AUC: 0.5
  Features: ['CPMGH2O_109.73048', 'CPMGH2O_117.85752', 'CPMGH2O_121.92104', 'CPMGH2O_130.04808', 'CPMGH2O_146.30216']
  Equation: 0.715
  Parameters: {'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19148dd10>, <gplearn.functions._Function object at 0x7fb1368798d0>, <gplearn.functions._Function object at 0x7fb135e9d6d0>, <gplearn.functions._Function object at 0x7fb135e9e310>]}


In [ ]:
#20250502 for swelling　スケーリング削除 カスタム関数シンプル化
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('Swelling_RDKit_DSC_MSEGPMGnodry_Top10_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', square_function, cbrt_function]
param_grid_improved = {
    'n_features_to_select': [3, 5, 10],
    'function_set': [function_set],
    'population_size': [500, 1000, 3000],
    'generations': [50, 100, 300],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.0001, 0.001, 0.01],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb13461a110>, <gplearn.functions._Function object at 0x7fb11d226d10>]}


<ipython-input-16-7b8fde7ea9e1>:49: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 2/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 500, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 3, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb13461a110>, <gplearn.functions._Function object at 0x7fb11d226d10>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb13461a110>, <gplearn.functions._Function object at 0x7fb11d226d10>]}


...


Trial 4/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 500, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 3, 'max_samples': 0.9, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb13461a110>, <gplearn.functions._Function object at 0x7fb11d226d10>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 5/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb13461a110>, <gplearn.functions._Function object at 0x7fb11d226d10>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.875
  Precision: 0.8333333333333334
  Recall: 1.0
  F1-score: 0.9090909090909091
  AUC: 1.0
  Features: ['MaxPartialCharge', 'NHOHCount', 'NumHDonors', 'CPMGH2O_109.73048', 'CPMGH2O_117.85752', 'CPMGH2O_121.92104', 'CPMGH2O_130.04808', 'CPMGH2O_146.30216', 'CPMGH2O_180.84208', 'CPMGH2O_256.0172']
  Equation: sqrt(mul(neg(cbrt(abs(sub(0.583, NumHDonors)))), sub(mul(cbrt(NHOHCount), -0.420), neg(sub(sub(mul(mul(mul(neg(CPMGH2O_146.30216), sub(CPMGH2O_109.73048, CPMGH2O_256.0172)), add(neg(CPMGH2O_109.73048), sqrt(MaxPartialCharge))), add(cbrt(-0.233), cbrt(CPMGH2O_256.0172))), add(mul(inv(CPMGH2O_121.92104), mul(-0.223, CPMGH2O_117.85752)), sqrt(-0.290))), neg(cbrt(abs(sub(0.583, NumHDonors)))))))))
  Parameters: {'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_sa

In [ ]:
#20250506 for swelling　スケーリング削除 カスタム関数シンプル化　再トライ
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('Swelling_RDKit_DSC_MSEGPMGnodry_Top10_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
# sqrt_function の定義は残しておいても問題ありませんが、function_set から削除します

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', log_function, 'inv', 'neg',
                square_function,
                multiply2_function, safe_divide_function,
                multiply3_function,
                exp_function]

param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [5000],
    'generations': [50],
    'tournament_size': [5],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.01],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 50, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7c76191e0110>, 'inv', 'neg', <gplearn.functions._Function object at 0x7c7619b0f050>, <gplearn.functions._Function object at 0x7c761c328390>, <gplearn.functions._Function object at 0x7c761a2cce90>, <gplearn.functions._Function object at 0x7c76191cff10>, <gplearn.functions._Function object at 0x7c76191cffd0>]}


<ipython-input-1-35244e505ac8>:46: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-1-35244e505ac8>:55: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-1-35244e505ac8>:55: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 1.0
  Precision: 1.0
  Recall: 1.0
  F1-score: 1.0
  AUC: 1.0
  Features: ['MaxPartialCharge', 'NHOHCount', 'NumHDonors', 'Mobile_water_109ms', 'Mobile_water_118ms', 'Mobile_water_122ms', 'Mobile_water_130ms', 'Mobile_water_146ms', 'Mobile_water_181ms', 'Mobile_water_256ms']
  Equation: exp(mul2(log(exp(log(neg(NHOHCount)))), Mobile_water_181ms))
  Parameters: {'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 50, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7c76191e0110>, 'inv', 'neg', <gplearn.functions._Function object at 0x7c7619b0f050>, <gplearn.functions._Function object at 0x7c761c328390>, <gplearn.functions._Function object at 0x7c761a2cce90>, <gplearn.functions._Function object at 0x7c7619

<ipython-input-1-35244e505ac8>:55: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-1-35244e505ac8>:55: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換


In [ ]:
#20250511 for swelling　スケーリング削除 カスタム関数シンプル化　再トライ
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('Swelling_RDKit_DSCafter0.6cut_MSEGPMGnodry_Top10_250511.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
# sqrt_function の定義は残しておいても問題ありませんが、function_set から削除します

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', log_function, 'inv', 'neg',
                square_function,
                multiply2_function, safe_divide_function,
                multiply3_function,
                exp_function]

param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [5000],
    'generations': [10, 50, 250, 500],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.01, 0.03, 0.05, 0.07],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

<ipython-input-2-2732d5c531f5>:46: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-2-2732d5c531f5>:55: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-2-2732d5c531f5>:55: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換



Trial 1/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.05, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7aaef2437190>, 'inv', 'neg', <gplearn.functions._Function object at 0x7aaf9c244e10>, <gplearn.functions._Function object at 0x7aaef12b36d0>, <gplearn.functions._Function object at 0x7aaef2435510>, <gplearn.functions._Function object at 0x7aaef2436f10>, <gplearn.functions._Function object at 0x7aaef2436650>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 2/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.07, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 50, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7aaef2437190>, 'inv', 'neg', <gplearn.functions._Function object at 0x7aaf9c244e10>, <gplearn.functions._Function object at 0x7aaef12b36d0>, <gplearn.functions._Function object at 0x7aaef2435510>, <gplearn.functions._Function object at 0x7aaef2436f10>, <gplearn.functions._Function object at 0x7aaef2436650>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7aaef2437190>, 'inv', 'neg', <gplearn.functions._Function object at 0x7aaf9c244e10>, <gplearn.functions._Function object at 0x7aaef12b36d0>, <gplearn.functions._Function object at 0x7aaef2435510>, <gplearn.functions._Function object at 0x7aaef2436f10>, <gplearn.functions._Function object at 0x7aaef2436650>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [ ]:
#20250502 for HSQC
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HSQC_RDKit_DSC_CPMGMSEnodry_Top10_Features_250510.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# 特徴量のスケーリング
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function] # 関数オブジェクトを直接指定

param_grid_improved = {
    'n_features_to_select': [5, 10, 20],
    'function_set': [function_set],
    'population_size': [4000],
    'generations': [300, 500],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.01],
}

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 4000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 20, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7c7604dde4d0>, <gplearn.functions._Function object at 0x7c75f8984490>, <gplearn.functions._Function object at 0x7c75f8985bd0>, <gplearn.functions._Function object at 0x7c75f8985450>]}


<ipython-input-3-9c2329a11484>:49: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 2/10 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 4000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7c7604dde4d0>, <gplearn.functions._Function object at 0x7c75f8984490>, <gplearn.functions._Function object at 0x7c75f8985bd0>, <gplearn.functions._Function object at 0x7c75f8985450>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 4000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7c7604dde4d0>, <gplearn.functions._Function object at 0x7c75f8984490>, <gplearn.functions._Function object at 0x7c75f8985bd0>, <gplearn.functions._Function object at 0x7c75f8985450>]}


...


Trial 9/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 4000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 20, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7c7604dde4d0>, <gplearn.functions._Function object at 0x7c75f8984490>, <gplearn.functions._Function object at 0x7c75f8985bd0>, <gplearn.functions._Function object at 0x7c75f8985450>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 10/10 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 4000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7c7604dde4d0>, <gplearn.functions._Function object at 0x7c75f8984490>, <gplearn.functions._Function object at 0x7c75f8985bd0>, <gplearn.functions._Function object at 0x7c75f8985450>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.75
  Precision: 1.0
  Recall: 0.5
  F1-score: 0.6666666666666666
  AUC: 0.75
  Features: ['PEOE_VSA14', 'SlogP_VSA1', 'fr_COO', 'fr_COO2', 'Mobile_water_93ms', 'Mobile_water_130ms', 'Mobile_water_158ms', 'Mobile_water_195ms', 'Mobile_water_375ms', 'Mobile_chains_191ms']
  Equation: abs(mul2(sub(Mobile_chains_191ms, fr_COO), -0.425))
  Parameters: {'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 4000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7c7604dde4d0>, <gplearn.functions._Function object at 0x7c75f8984490>, <gplearn.functions._Function object at 0x7c75f8985bd0>, <gplearn.functions._Function object at 0x7c75f8985450>]}


In [ ]:
#20250503 for HSQC - スケーリング削除 カスタム関数シンプル化
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HSQC_RDKit_DSC_CPMGMSEnodry_Top10_Features_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', square_function, cbrt_function]
param_grid_improved = {
    'n_features_to_select': [3, 5, 10],
    'function_set': [function_set],
    'population_size': [500, 1000, 3000],
    'generations': [50, 100, 300],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.0001, 0.001, 0.01],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

<ipython-input-4-6cac57b1c934>:41: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b37c911de50>, <gplearn.functions._Function object at 0x7b37bf251150>]}

Trial 2/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 500, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 3, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b37c911de50>, <gplearn.functions._Function object at 0x7b37bf251150>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b37c911de50>, <gplearn.functions._Function object at 0x7b37bf251150>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 4/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 500, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 3, 'max_samples': 0.9, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b37c911de50>, <gplearn.functions._Function object at 0x7b37bf251150>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 5/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b37c911de50>, <gplearn.functions._Function object at 0x7b37bf251150>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.5
  Precision: 0.5
  Recall: 0.25
  F1-score: 0.3333333333333333
  AUC: 0.1875
  Features: ['PEOE_VSA14', 'CPMGH2O_158.49272', 'CPMGH2O_195.0644']
  Equation: mul(abs(mul(abs(sub(sqrt(PEOE_VSA14), sqrt(0.856))), sqrt(sub(log(add(cbrt(cbrt(CPMGH2O_195.0644)), CPMGH2O_158.49272)), cbrt(square(abs(sqrt(mul(sqrt(square(log(neg(PEOE_VSA14)))), sqrt(sub(log(add(cbrt(cbrt(CPMGH2O_195.0644)), sqrt(sub(log(add(cbrt(abs(sub(sqrt(PEOE_VSA14), sqrt(0.856)))), CPMGH2O_158.49272)), cbrt(inv(PEOE_VSA14)))))), log(PEOE_VSA14)))))))))))), abs(sub(square(abs(log(PEOE_VSA14))), cbrt(log(CPMGH2O_158.49272)))))
  Parameters: {'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 500, 'parsimony_coefficient': 0.0001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 3, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gp

In [ ]:
#20250502 for HSQC　スケーリング削除 カスタム関数シンプル化2
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HSQC_RDKit_DSC_CPMGMSEnodry_Top10_Features_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', square_function, cbrt_function]
param_grid_improved = {
    'n_features_to_select': [5, 10],
    'function_set': [function_set],
    'population_size': [5000],
    'generations': [50, 100, 300],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.01],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b379d8a1c10>, <gplearn.functions._Function object at 0x7b379c91ff90>]}


<ipython-input-6-13fd8987bcba>:49: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 2/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b379d8a1c10>, <gplearn.functions._Function object at 0x7b379c91ff90>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 50, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b379d8a1c10>, <gplearn.functions._Function object at 0x7b379c91ff90>]}


...


Trial 4/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b379d8a1c10>, <gplearn.functions._Function object at 0x7b379c91ff90>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 5/5 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b379d8a1c10>, <gplearn.functions._Function object at 0x7b379c91ff90>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.375
  Precision: 0.0
  Recall: 0.0
  F1-score: 0.0
  AUC: 0.46875
  Features: ['PEOE_VSA14', 'SlogP_VSA1', 'fr_COO', 'fr_COO2', 'CPMGH2O_93.4764', 'CPMGH2O_130.04808', 'CPMGH2O_158.49272', 'CPMGH2O_195.0644', 'CPMGH2O_375.89104', 'CPMGD2O_191.00088']
  Equation: abs(abs(abs(log(PEOE_VSA14))))
  Parameters: {'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7b379d8a1c10>, <gplearn.functions._Function object at 0x7b379c91ff90>]}


In [ ]:
#20250506 for HSQC　スケーリング削除 カスタム関数シンプル化　再トライ
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HSQC_RDKit_DSC_CPMGMSEnodry_Top10_Features_250502.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
# sqrt_function の定義は残しておいても問題ありませんが、function_set から削除します

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', log_function, 'inv', 'neg',
                square_function,
                multiply2_function, safe_divide_function,
                multiply3_function,
                exp_function]

param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [5000],
    'generations': [50],
    'tournament_size': [5],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.01],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

<ipython-input-7-7366e200fdea>:46: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-7-7366e200fdea>:55: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-7-7366e200fdea>:55: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 50, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x78a0378bdcd0>, 'inv', 'neg', <gplearn.functions._Function object at 0x78a054c33fd0>, <gplearn.functions._Function object at 0x78a054c33210>, <gplearn.functions._Function object at 0x78a0378ec110>, <gplearn.functions._Function object at 0x78a0378ef050>, <gplearn.functions._Function object at 0x78a03654df90>]}

Best Model Summary:
  Accuracy: 0.375
  Precision: 0.0
  Recall: 0.0
  F1-score: 0.0
  AUC: 0.46875
  Features: ['PEOE_VSA14', 'SlogP_VSA1', 'fr_COO', 'fr_COO2', 'CPMGH2O_93.4764', 'CPMGH2O_130.04808', 'CPMGH2O_158.49272', 'CPMGH2O_195.0644', 'CPMGH2O_375.89104', 'CPMGD2O_191.00088']
  Equation: abs(

In [ ]:
#20250502 for HNCO
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# 特徴量のスケーリング
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function] # 関数オブジェクトを直接指定

param_grid_improved = {
    'n_features_to_select': [3, 5, 10],
    'function_set': [function_set],
    'population_size': [300, 1000, 3000],
    'generations': [300, 500],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.01, 0.1],
}

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 3, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fc3ec1bae50>, <gplearn.functions._Function object at 0x7fc3c4507610>, <gplearn.functions._Function object at 0x7fc3cc8d7110>, <gplearn.functions._Function object at 0x7fc3bfc95790>]}


<ipython-input-2-38685a8f7511>:49: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 2/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fc3ec1bae50>, <gplearn.functions._Function object at 0x7fc3c4507610>, <gplearn.functions._Function object at 0x7fc3cc8d7110>, <gplearn.functions._Function object at 0x7fc3bfc95790>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 300, 'parsimony_coefficient': 0.1, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fc3ec1bae50>, <gplearn.functions._Function object at 0x7fc3c4507610>, <gplearn.functions._Function object at 0x7fc3cc8d7110>, <gplearn.functions._Function object at 0x7fc3bfc95790>]}


...

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 10/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 300, 'parsimony_coefficient': 0.1, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fc3ec1bae50>, <gplearn.functions._Function object at 0x7fc3c4507610>, <gplearn.functions._Function object at 0x7fc3cc8d7110>, <gplearn.functions._Function object at 0x7fc3bfc95790>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.75
  Precision: 0.6666666666666666
  Recall: 0.6666666666666666
  F1-score: 0.6666666666666666
  AUC: 0.7666666666666666
  Features: ['BCUT2D_MWHI', 'PEOE_VSA13', 'VSA_EState10', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'DSC_area', 'DSC_peak_height', 'MSEH2O_0.12992', 'CPMGD2O_119.88928']
  Equation: abs(mul2(fr_Al_COO, VSA_EState10))
  Parameters: {'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fc3ec1bae50>, <gplearn.functions._Function object at 0x7fc3c4507610>, <gplearn.functions._Function object at 0x7fc3cc8d7110>, <gplearn.functions._Function object at 0x7fc3bfc95790>]}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#20250502 for HNCO - スケーリング除去
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function] # 関数オブジェクトを直接指定

param_grid_improved = {
    'n_features_to_select': [5, 10],
    'function_set': [function_set],
    'population_size': [2000, 3000],
    'generations': [300, 500],
    'tournament_size': [5, 10],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.03],
    'p_point_mutation': [0.05],
    'max_samples': [0.9],
    'parsimony_coefficient': [0.001, 0.01],
    'max_depth': [5, 7, 10], # max_depth を追加
}

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 2000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'max_depth': 10, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19c1b8450>, <gplearn.functions._Function object at 0x7fb19ae057d0>, <gplearn.functions._Function object at 0x7fb1e8640e90>, <gplearn.functions._Function object at 0x7fb19beaedd0>]}


<ipython-input-1-226ffadf6cf7>:41: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 2/10 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 2000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'max_depth': 7, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19c1b8450>, <gplearn.functions._Function object at 0x7fb19ae057d0>, <gplearn.functions._Function object at 0x7fb1e8640e90>, <gplearn.functions._Function object at 0x7fb19beaedd0>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/10 with parameters:
{'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 2000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'max_depth': 7, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19c1b8450>, <gplearn.functions._Function object at 0x7fb19ae057d0>, <gplearn.functions._Function object at 0x7fb1e8640e90>, <gplearn.functions._Function object at 0x7fb19beaedd0>]}


...


Trial 9/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 2000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 0.9, 'max_depth': 5, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19c1b8450>, <gplearn.functions._Function object at 0x7fb19ae057d0>, <gplearn.functions._Function object at 0x7fb1e8640e90>, <gplearn.functions._Function object at 0x7fb19beaedd0>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 10/10 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 3000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'max_depth': 5, 'generations': 300, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19c1b8450>, <gplearn.functions._Function object at 0x7fb19ae057d0>, <gplearn.functions._Function object at 0x7fb1e8640e90>, <gplearn.functions._Function object at 0x7fb19beaedd0>]}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.75
  Precision: 0.6666666666666666
  Recall: 0.6666666666666666
  F1-score: 0.6666666666666666
  AUC: 0.7666666666666666
  Features: ['BCUT2D_MWHI', 'PEOE_VSA13', 'VSA_EState10', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'DSC_area', 'DSC_peak_height', 'MSEH2O_0.12992', 'CPMGD2O_119.88928']
  Equation: abs(mul2(fr_Al_COO, VSA_EState10))
  Parameters: {'tournament_size': 10, 'stopping_criteria': 0.05, 'population_size': 2000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.03, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 0.9, 'max_depth': 7, 'generations': 500, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7fb19c1b8450>, <gplearn.functions._Function object at 0x7fb19ae057d0>, <gplearn.functions._Function object at 0x7fb1e8640e90>, <gplearn.functions._Function object at 0x7fb19beaedd0>]}


In [ ]:
#20250502 for HNCO - スケーリング除去, カスタム関数複雑化

import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def protected_cube(x):
    return np.where(np.abs(x) < 1e10, x**3, np.sign(x) * 1e10)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), np.inf)

tanh_function = make_function(function=np.tanh, arity=1, name='tanh')

def protected_sigmoid(x):
    return 1 / (1 + np.exp(-x))

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
cube_function = make_function(function=protected_cube, arity=1, name='cube')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
sigmoid_function = make_function(function=protected_sigmoid, arity=1, name='sigmoid')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function,
                cube_function, multiply3_function,
                exp_function, tanh_function,
                sigmoid_function] # 全てのカスタム関数を追加

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

FileNotFoundError: [Errno 2] No such file or directory: 'HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv'

In [ ]:
#20250502 for HNCO - スケーリング除去, カスタム関数複雑化2

import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def protected_cube(x):
    return np.where(np.abs(x) < 1e10, x**3, np.sign(x) * 1e10)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), np.inf)

tanh_function = make_function(function=np.tanh, arity=1, name='tanh')

def protected_sigmoid(x):
    return 1 / (1 + np.exp(-x))

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
cube_function = make_function(function=protected_cube, arity=1, name='cube')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
sigmoid_function = make_function(function=protected_sigmoid, arity=1, name='sigmoid')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function,
                cube_function, multiply3_function,
                exp_function, tanh_function,
                sigmoid_function] # 全てのカスタム関数を追加

param_grid_improved = {
    'n_features_to_select': [20],
    'function_set': [function_set],
    'population_size': [1000],
    'generations': [20],
    'tournament_size': [20],
    'stopping_criteria': [0.0],
    'p_crossover': [0.9],
    'p_subtree_mutation': [0.01],
    'p_hoist_mutation': [0.01],
    'p_point_mutation': [0.01],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.001],
 }

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")


Trial 1/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 1000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.01, 'p_point_mutation': 0.01, 'p_hoist_mutation': 0.01, 'p_crossover': 0.9, 'n_features_to_select': 20, 'max_samples': 1.0, 'generations': 20, 'function_set': ['add', 'sub', 'mul', 'abs', 'sqrt', 'log', 'inv', 'neg', <gplearn.functions._Function object at 0x7f1d0c06fd90>, <gplearn.functions._Function object at 0x7f1d0c06f8d0>, <gplearn.functions._Function object at 0x7f1d0c06f190>, <gplearn.functions._Function object at 0x7f1cf4cbf490>, <gplearn.functions._Function object at 0x7f1cf9bace10>, <gplearn.functions._Function object at 0x7f1d0c06f590>, <gplearn.functions._Function object at 0x7f1d002c3e50>, <gplearn.functions._Function object at 0x7f1d0507e7d0>, <gplearn.functions._Function object at 0x7f1d002c3850>]}


<ipython-input-4-c75a933603cd>:42: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
<ipython-input-4-c75a933603cd>:51: RuntimeWarning: overflow encountered in exp
  return np.where(x < 100, np.exp(x), np.inf)
<ipython-input-4-c75a933603cd>:51: RuntimeWarning: overflow encountered in exp
  return np.where(x < 100, np.exp(x), np.inf)
<ipython-input-4-c75a933603cd>:51: RuntimeWarning: overflow encountered in 

ValueError: Input contains NaN.

In [ ]:
#20250502 for HNCO - スケーリング除去, カスタム関数複雑化2→修正版

import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込みと NaN 処理
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# NaN のチェック
print("NaNを含む特徴量の数:\n", X.isnull().sum())
print("NaNを含むターゲット変数の数:", y.isnull().sum())

# NaN の処理 (平均値で補完)
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def protected_cube(x):
    return np.where(np.abs(x) < 1e10, x**3, np.sign(x) * 1e10)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

def protected_sqrt(x):
    return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換

def protected_tanh(x):
    return np.tanh(x)

def protected_sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -100, 100))) # オーバーフロー対策としてクリップ

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
cube_function = make_function(function=protected_cube, arity=1, name='cube')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
sqrt_function = make_function(function=protected_sqrt, arity=1, name='sqrt')
tanh_function = make_function(function=protected_tanh, arity=1, name='tanh')
sigmoid_function = make_function(function=protected_sigmoid, arity=1, name='sigmoid')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', sqrt_function, log_function, 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function,
                cube_function, multiply3_function,
                exp_function, tanh_function,
                sigmoid_function]

param_grid_improved = {
    'n_features_to_select': [5],
    'function_set': [function_set],
    'population_size': [1000],
    'generations': [20],
    'tournament_size': [20],
    'stopping_criteria': [0.0],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.001], # 少し大きめの値を設定
}

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

NaNを含む特徴量の数:
 BCUT2D_MWHI          0
Ipc                  0
PEOE_VSA13           0
EState_VSA5          0
VSA_EState10         0
fr_Al_COO            0
fr_Al_OH             0
fr_Al_OH_noTert      0
DSC_area             0
DSC_peak_height      0
MSEH2O_0.0992        0
MSEH2O_0.12992       0
MSEH2O_0.14528       0
MSEH2O_0.15424       0
MSED2O_0.0992        0
CPMGH2O_243.82664    0
CPMGD2O_46.74592     0
CPMGD2O_93.4764      0
CPMGD2O_119.88928    0
CPMGD2O_264.14424    0
dtype: int64
NaNを含むターゲット変数の数: 0

Trial 1/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 1000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 1.0, 'generations': 20, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1d2baa1010>, <gplearn.functions._Function object at 0x7f1d2baa0d90>, 'inv', 'neg', <gplearn.functions._

<ipython-input-10-991b7af3ef6b>:50: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-10-991b7af3ef6b>:62: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-10-991b7af3ef6b>:62: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-10-991b7af3ef6b>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_d


Best Model Summary:
  Accuracy: 0.625
  Precision: 0.5
  Recall: 0.6666666666666666
  F1-score: 0.5714285714285714
  AUC: 0.5333333333333333
  Features: ['BCUT2D_MWHI', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'DSC_peak_height']
  Equation: square(sigmoid(sub(sub(abs(DSC_peak_height), sub(square(fr_Al_OH), sub(abs(DSC_peak_height), sub(square(sigmoid(sub(div(inv(BCUT2D_MWHI), cbrt(fr_Al_OH)), sqrt(0.894)))), div(inv(BCUT2D_MWHI), cbrt(fr_Al_OH)))))), 0.894)))
  Parameters: {'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 1000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 5, 'max_samples': 1.0, 'generations': 20, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1d2baa1010>, <gplearn.functions._Function object at 0x7f1d2baa0d90>, 'inv', 'neg', <gplearn.functions._Function object at 0x7f1cf004a9d0>, <gplearn.function

In [ ]:
#20250502 for HNCO - スケーリング除去, カスタム関数複雑化2→修正版2

import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込みと NaN 処理
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# NaN のチェック
print("NaNを含む特徴量の数:\n", X.isnull().sum())
print("NaNを含むターゲット変数の数:", y.isnull().sum())

# NaN の処理 (平均値で補完)
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def protected_cube(x):
    return np.where(np.abs(x) < 1e10, x**3, np.sign(x) * 1e10)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

def protected_sqrt(x):
    return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
cube_function = make_function(function=protected_cube, arity=1, name='cube')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
sqrt_function = make_function(function=protected_sqrt, arity=1, name='sqrt')
tanh_function = make_function(function=protected_tanh, arity=1, name='tanh')
sigmoid_function = make_function(function=protected_sigmoid, arity=1, name='sigmoid')

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', sqrt_function, log_function, 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function,
                cube_function, multiply3_function,
                exp_function]

param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [1000, 3000, 5000],
    'generations': [20, 100],
    'tournament_size': [20],
    'stopping_criteria': [0.0],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.001], # 少し大きめの値を設定
}

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

NaNを含む特徴量の数:
 BCUT2D_MWHI          0
Ipc                  0
PEOE_VSA13           0
EState_VSA5          0
VSA_EState10         0
fr_Al_COO            0
fr_Al_OH             0
fr_Al_OH_noTert      0
DSC_area             0
DSC_peak_height      0
MSEH2O_0.0992        0
MSEH2O_0.12992       0
MSEH2O_0.14528       0
MSEH2O_0.15424       0
MSED2O_0.0992        0
CPMGH2O_243.82664    0
CPMGD2O_46.74592     0
CPMGD2O_93.4764      0
CPMGD2O_119.88928    0
CPMGD2O_264.14424    0
dtype: int64
NaNを含むターゲット変数の数: 0

Trial 1/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 1000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 20, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1ced9168d0>, <gplearn.functions._Function object at 0x7f1ced917a50>, 'inv', 'neg', <gplearn.functions.

<ipython-input-33-3785316c0881>:50: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-33-3785316c0881>:62: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-33-3785316c0881>:62: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_d


Trial 2/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 3000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 20, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1ced9168d0>, <gplearn.functions._Function object at 0x7f1ced917a50>, 'inv', 'neg', <gplearn.functions._Function object at 0x7f1cf82101d0>, <gplearn.functions._Function object at 0x7f1cefe80310>, <gplearn.functions._Function object at 0x7f1cf18e7210>, <gplearn.functions._Function object at 0x7f1cf7931e10>, <gplearn.functions._Function object at 0x7f1ced9174d0>, <gplearn.functions._Function object at 0x7f1ced916310>, <gplearn.functions._Function object at 0x7f1ced915c90>]}


<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 3/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 5000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 20, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1ced9168d0>, <gplearn.functions._Function object at 0x7f1ced917a50>, 'inv', 'neg', <gplearn.functions._Function object at 0x7f1cf82101d0>, <gplearn.functions._Function object at 0x7f1cefe80310>, <gplearn.functions._Function object at 0x7f1cf18e7210>, <gplearn.functions._Function object at 0x7f1cf7931e10>, <gplearn.functions._Function object at 0x7f1ced9174d0>, <gplearn.functions._Function object at 0x7f1ced916310>, <gplearn.functions._Function object at 0x7f1ced915c90>]}


...

<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 5/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 3000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1ced9168d0>, <gplearn.functions._Function object at 0x7f1ced917a50>, 'inv', 'neg', <gplearn.functions._Function object at 0x7f1cf82101d0>, <gplearn.functions._Function object at 0x7f1cefe80310>, <gplearn.functions._Function object at 0x7f1cf18e7210>, <gplearn.functions._Function object at 0x7f1cf7931e10>, <gplearn.functions._Function object at 0x7f1ced9174d0>, <gplearn.functions._Function object at 0x7f1ced916310>, <gplearn.functions._Function object at 0x7f1ced915c90>]}


<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 6/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 5000, 'parsimony_coefficient': 0.001, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7f1ced9168d0>, <gplearn.functions._Function object at 0x7f1ced917a50>, 'inv', 'neg', <gplearn.functions._Function object at 0x7f1cf82101d0>, <gplearn.functions._Function object at 0x7f1cefe80310>, <gplearn.functions._Function object at 0x7f1cf18e7210>, <gplearn.functions._Function object at 0x7f1cf7931e10>, <gplearn.functions._Function object at 0x7f1ced9174d0>, <gplearn.functions._Function object at 0x7f1ced916310>, <gplearn.functions._Function object at 0x7f1ced915c90>]}

Best Model Summary:
  Accuracy: 0.75
  Precision: 0.6666666666666666
  Recall: 0.6666666666666666
  F1-score: 0.6666666666666666
 

<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換
<ipython-input-33-3785316c0881>:65: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x >= 0, np.sqrt(x), 0) # 負の値を 0 に置換


In [ ]:
#20250502 for HNCO - スケーリング除去, カスタム関数複雑化2→修正版3

import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込みと NaN 処理
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# NaN のチェック
print("NaNを含む特徴量の数:\n", X.isnull().sum())
print("NaNを含むターゲット変数の数:", y.isnull().sum())

# NaN の処理 (平均値で補完)
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def protected_cbrt(x):
    return np.cbrt(x)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def protected_cube(x):
    return np.where(np.abs(x) < 1e10, x**3, np.sign(x) * 1e10)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
cbrt_function = make_function(function=protected_cbrt, arity=1, name='cbrt')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
cube_function = make_function(function=protected_cube, arity=1, name='cube')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
# sqrt_function の定義は残しておいても問題ありませんが、function_set から削除します

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', log_function, 'inv', 'neg',
                square_function, cbrt_function,
                multiply2_function, safe_divide_function,
                cube_function, multiply3_function,
                exp_function]

param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [1000],
    'generations': [100],
    'tournament_size': [20],
    'stopping_criteria': [0.0],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.01], # 少し大きめの値を設定
}

# パラメータサンプリング
n_iter = 10
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train, y_train) # スケール前のデータを使用
    X_test_sel = selector.transform(X_test) # スケール前のデータを使用
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

NaNを含む特徴量の数:
 BCUT2D_MWHI          0
Ipc                  0
PEOE_VSA13           0
EState_VSA5          0
VSA_EState10         0
fr_Al_COO            0
fr_Al_OH             0
fr_Al_OH_noTert      0
DSC_area             0
DSC_peak_height      0
MSEH2O_0.0992        0
MSEH2O_0.12992       0
MSEH2O_0.14528       0
MSEH2O_0.15424       0
MSED2O_0.0992        0
CPMGH2O_243.82664    0
CPMGD2O_46.74592     0
CPMGD2O_93.4764      0
CPMGD2O_119.88928    0
CPMGD2O_264.14424    0
dtype: int64
NaNを含むターゲット変数の数: 0

Trial 1/10 with parameters:
{'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7dc4fce18950>, 'inv', 'neg', <gplearn.functions._Function object at 0x7dc54c92aed0>, <gplearn.functions.

<ipython-input-3-f847ce53b390>:50: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-3-f847ce53b390>:62: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-3-f847ce53b390>:62: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Best Model Summary:
  Accuracy: 0.75
  Precision: 0.6666666666666666
  Recall: 0.6666666666666666
  F1-score: 0.6666666666666666
  AUC: 0.7
  Features: ['BCUT2D_MWHI', 'PEOE_VSA13', 'VSA_EState10', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'DSC_area', 'DSC_peak_height', 'MSEH2O_0.12992', 'CPMGD2O_119.88928']
  Equation: abs(mul2(square(cbrt(VSA_EState10)), fr_Al_COO))
  Parameters: {'tournament_size': 20, 'stopping_criteria': 0.0, 'population_size': 1000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 100, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x7dc4fce18950>, 'inv', 'neg', <gplearn.functions._Function object at 0x7dc54c92aed0>, <gplearn.functions._Function object at 0x7dc4fd500cd0>, <gplearn.functions._Function object at 0x7dc4fd7d7f90>, <gplearn.functions._Function object at 0x7dc54c71a410>

In [ ]:
#20250506 for HNCO　スケーリング削除 カスタム関数シンプル化　再トライ
import pandas as pd
import numpy as np
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split, ParameterSampler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.preprocessing import StandardScaler # 削除
import random
from gplearn.functions import make_function

# ランダムシード固定
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

# データ読み込み
data = pd.read_csv('HNCO_RDKit_DSC_MSECPMGdryTop10_Features.csv')
if 'Unnamed: 0' in data.columns:
    X = data.drop(['CA1', 'Unnamed: 0'], axis=1)
else:
    X = data.drop('CA1', axis=1)
y = data['CA1']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed, stratify=y
)

# # 特徴量のスケーリング # 削除
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled_df = X_train.copy() # スケーリングしない場合は元のDataFrameをコピー
X_test_scaled_df = X_test.copy()   # スケーリングしない場合は元のDataFrameをコピー

# カスタム関数定義 (NaN 対策を含む)
def protected_square(x):
    return np.where(np.abs(x) < 1e10, x**2, 1e10)

def multiply2(x1, x2):
    return x1 * x2

def safe_divide(x1, x2):
    return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)

def multiply3(x1, x2, x3):
    return x1 * x2 * x3

def protected_exp(x):
    return np.where(x < 100, np.exp(x), 1e10) # 無限大を大きな有限数に置換

def protected_log(x):
    return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換

# make_function を使って GPlearn が認識できる形式に変換
square_function = make_function(function=protected_square, arity=1, name='square')
multiply2_function = make_function(function=multiply2, arity=2, name='mul2')
safe_divide_function = make_function(function=safe_divide, arity=2, name='div')
multiply3_function = make_function(function=multiply3, arity=3, name='mul3')
exp_function = make_function(function=protected_exp, arity=1, name='exp')
log_function = make_function(function=protected_log, arity=1, name='log')
# sqrt_function の定義は残しておいても問題ありませんが、function_set から削除します

# 改善版：関数セットとパラメータ
function_set = ['add', 'sub', 'mul', 'abs', log_function, 'inv', 'neg',
                square_function,
                multiply2_function, safe_divide_function,
                multiply3_function,
                exp_function]

param_grid_improved = {
    'n_features_to_select': [10],
    'function_set': [function_set],
    'population_size': [5000],
    'generations': [200],
    'tournament_size': [5],
    'stopping_criteria': [0.05],
    'p_crossover': [0.85],
    'p_subtree_mutation': [0.05],
    'p_hoist_mutation': [0.05],
    'p_point_mutation': [0.05],
    'max_samples': [1.0],
    'parsimony_coefficient': [0.01],
   }

# パラメータサンプリング
n_iter = 5
param_sampler = ParameterSampler(param_grid_improved, n_iter=n_iter, random_state=random_seed)

best_score = -1.0
best_params = None
best_estimator = None
all_results = []

for i, params in enumerate(param_sampler):
    print(f"\nTrial {i+1}/{n_iter} with parameters:\n{params}")

    # 特徴量選択
    selector = SelectKBest(score_func=f_regression, k=params['n_features_to_select'])
    X_train_sel = selector.fit_transform(X_train_scaled_df, y_train)
    X_test_sel = selector.transform(X_test_scaled_df)
    selected_features = X_train.columns[selector.get_support(indices=True)].tolist()

    # モデル構築
    est = SymbolicRegressor(
        random_state=random_seed,
        function_set=params['function_set'],
        metric='mse',
        population_size=params['population_size'],
        generations=params['generations'],
        tournament_size=params['tournament_size'],
        stopping_criteria=params['stopping_criteria'],
        p_crossover=params['p_crossover'],
        p_subtree_mutation=params['p_subtree_mutation'],
        p_hoist_mutation=params['p_hoist_mutation'],
        p_point_mutation=params['p_point_mutation'],
        max_samples=params['max_samples'],
        parsimony_coefficient=params['parsimony_coefficient'],
        n_jobs=-1,
        feature_names=selected_features
    )

    est.fit(X_train_sel, y_train)
    y_pred = est.predict(X_test_sel)
    y_pred_binary = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    all_results.append({
        'params': params,
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'r2_score': r2,
        'features': selected_features,
        'equation': str(est._program)
    })

    if acc > best_score:
        best_score = acc
        best_params = params
        best_estimator = est
        best_features = selected_features
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        best_auc = auc

# ベストモデル結果
print("\nBest Model Summary:")
print(f"  Accuracy: {best_score}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  F1-score: {best_f1}")
print(f"  AUC: {best_auc}")
print(f"  Features: {best_features}")
print(f"  Equation: {best_estimator._program}")
print(f"  Parameters: {best_params}")

<ipython-input-11-dd360756b308>:46: RuntimeWarning: invalid value encountered in divide
  return np.where(np.abs(x2) < 1e-6, 1.0, x1 / x2)
<ipython-input-11-dd360756b308>:55: RuntimeWarning: divide by zero encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
<ipython-input-11-dd360756b308>:55: RuntimeWarning: invalid value encountered in log
  return np.where(x > 0, np.log(x), -1e10) # 非正の値を大きな負の数に置換
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Trial 1/5 with parameters:
{'tournament_size': 5, 'stopping_criteria': 0.05, 'population_size': 5000, 'parsimony_coefficient': 0.01, 'p_subtree_mutation': 0.05, 'p_point_mutation': 0.05, 'p_hoist_mutation': 0.05, 'p_crossover': 0.85, 'n_features_to_select': 10, 'max_samples': 1.0, 'generations': 200, 'function_set': ['add', 'sub', 'mul', 'abs', <gplearn.functions._Function object at 0x78a04dc31750>, 'inv', 'neg', <gplearn.functions._Function object at 0x78a03597da50>, <gplearn.functions._Function object at 0x789fffd78f10>, <gplearn.functions._Function object at 0x789ff8ccdb10>, <gplearn.functions._Function object at 0x789ff8ccfc90>, <gplearn.functions._Function object at 0x78a04dc33010>]}

Best Model Summary:
  Accuracy: 0.75
  Precision: 0.6666666666666666
  Recall: 0.6666666666666666
  F1-score: 0.6666666666666666
  AUC: 0.7333333333333333
  Features: ['BCUT2D_MWHI', 'PEOE_VSA13', 'VSA_EState10', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'DSC_area', 'DSC_peak_height', 'MSEH2O_0.12